<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/11_car_price/car_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Ermittlung der Autokosten**

Ein Gebrauchtwagenhändler entwickelt eine App zur Neukundengewinnung. Darin erfahren Sie schnell den Marktwert Ihres Autos. Zu Ihrer Verfügung stehen historische Daten: technische Daten, Ausstattung und Preise von Autos. Sie müssen ein Modell bauen, um die Kosten zu ermitteln.

Der Kunde ist wichtig:


*   Vorhersagequalität;
*   Vorhersagegeschwindigkeit;
*   Studienzeit.

# Datenaufbereitung

In [1]:
# Wir importieren die notwendigen Bibliotheken
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression

In [2]:
# Wir laden Tabellen mit Daten
df = pd.read_csv('/content/autos.csv')

In [3]:
# Wir werden die erhaltenen Daten studieren
df.info()
display(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108886 entries, 0 to 108885
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        108886 non-null  object
 1   Price              108886 non-null  int64 
 2   VehicleType        97470 non-null   object
 3   RegistrationYear   108886 non-null  int64 
 4   Gearbox            102847 non-null  object
 5   Power              108886 non-null  int64 
 6   Model              102848 non-null  object
 7   Kilometer          108886 non-null  int64 
 8   RegistrationMonth  108886 non-null  int64 
 9   FuelType           98725 non-null   object
 10  Brand              108886 non-null  object
 11  NotRepaired        86960 non-null   object
 12  DateCreated        108886 non-null  object
 13  NumberOfPictures   108886 non-null  int64 
 14  PostalCode         108886 non-null  int64 
 15  LastSeen           108886 non-null  object
dtypes: int64(7), object(

(108886, 16)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Wir werden Spalten entfernen, die für die Analyse nicht benötigt werden. Die preisbestimmenden Bedingungen sind unserer Meinung nach: Baujahr, Kilometerstand, Marke und Modell und ob das Auto repariert wurde. Es gibt viele Modelle, aber wir haben Daten über die Leistung von Autos. Um die Modellarbeit zu beschleunigen, nehmen Sie an, dass eine Marke nur ein Modell mit einer bestimmten Leistung haben kann, was auch den Autotyp, den Getriebetyp und den Kraftstofftyp bestimmt.

In [4]:
df.drop(['DateCrawled', 'VehicleType', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 
         'LastSeen', 'Model', 'Gearbox', 'FuelType'], axis=1, inplace = True)

display(df.info())
display(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108886 entries, 0 to 108885
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             108886 non-null  int64 
 1   RegistrationYear  108886 non-null  int64 
 2   Power             108886 non-null  int64 
 3   Kilometer         108886 non-null  int64 
 4   Brand             108886 non-null  object
 5   NotRepaired       86960 non-null   object
dtypes: int64(4), object(2)
memory usage: 5.0+ MB


None

(108886, 6)

,Price,RegistrationYear,Power,Kilometer,Brand,NotRepaired
0,480,1993,0,150000,volkswagen,NaN
1,18300,2011,190,125000,audi,yes
2,9800,2004,163,125000,jeep,NaN
3,1500,2001,75,150000,volkswagen,no
4,3600,2008,69,90000,skoda,no


In [5]:
# Wir werden kategoriale Merkmale durch numerische ersetzen
df_ohe = pd.get_dummies(df, drop_first=True)
display(df_ohe.shape)
display(df_ohe.head())

(108886, 44)

,Price,RegistrationYear,Power,Kilometer,Brand_audi,Brand_bmw,Brand_chevrolet,Brand_chrysler,Brand_citroen,Brand_dacia,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
0,480,1993,0,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,18300,2011,190,125000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,3600,2008,69,90000,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [6]:
# Wir werden die Zielmerkmale und -bedingungen hervorheben
target = df_ohe['Price']
features = df_ohe.drop('Price', axis=1)

In [7]:
# Wir teilen die Daten in Stichproben auf
features_train_1, features_valid, target_train_1, target_valid = train_test_split(features, target, test_size=0.20, random_state=12345)
features_train, features_test, target_train, target_test = train_test_split(features_train_1, 
                                                                            target_train_1, test_size=0.25, random_state=12345)

In [16]:
# Wir prüfen, ob die Daten korrekt verteilt werden
print('{:.0%}'.format(features_train.shape[0]/features.shape[0]))
print('{:.0%}'.format(features_valid.shape[0]/features.shape[0]))
print('{:.0%}'.format(features_test.shape[0]/features.shape[0]))

60%
20%
20%


# Modelltraining

In [17]:
%%time
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid) 
result = mean_squared_error(target_valid, predictions) ** 0.5 
print("LinearRegression", result)

LinearRegression 3831.030829476241
CPU times: user 201 ms, sys: 115 ms, total: 316 ms
Wall time: 301 ms


In [18]:
best_result = 5000
best_depth = 0
for depth in range(10, 20, 1):
    model = DecisionTreeRegressor(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = mean_squared_error(target_valid, predictions_valid) ** 0.5
    if result < best_result:
        best_result = result
        best_depth = depth
    
print(best_depth, best_result)

12 2206.750286213146


In [20]:
%%time
model = DecisionTreeRegressor(random_state=12345, max_depth=12) 
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = mean_squared_error(target_valid, predictions) ** 0.5
print('DecisionTreeRegressor', result)

DecisionTreeRegressor 2206.750286213146
CPU times: user 398 ms, sys: 2.07 ms, total: 400 ms
Wall time: 415 ms


In [30]:
best_result = 5000
best_est = 0
best_depth = 0
for est in range(10, 41, 10):
    for depth in range (16, 18, 1):
        model = RandomForestRegressor(random_state=12345, n_estimators=90, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = mean_squared_error(target_valid, predictions_valid)**0.5 
        if result < best_result:
            best_result = result
            best_est = est
            best_depth = depth
            
print(best_est, best_depth, best_result)

10 17 2024.021517748526


In [31]:
%%time
model = RandomForestRegressor(random_state=12345, n_estimators=10, max_depth=17)
model.fit(features_train, target_train) 
predictions = model.predict(features_valid) 
result = mean_squared_error(target_valid, predictions)**0.5 
print('RandomForestRegressor', result)

RandomForestRegressor 2054.8847124616395
CPU times: user 2.99 s, sys: 20 ms, total: 3.01 s
Wall time: 3.42 s


In [32]:
best_result = 5000
best_est = 0
best_depth = 0
for est in range(2000, 2201, 100):
    for depth in range (9, 12, 1):
        model = LGBMRegressor(n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = mean_squared_error(target_valid, predictions_valid)**0.5 
        if result < best_result:
            best_result = result
            best_est = est
            best_depth = depth
            
print(best_est, best_depth, best_result)

NameError: ignored